In [30]:
import leidenalg
import igraph as ig
import json
import numpy as np
import copy
#from Levenshtein import distance as lev

In [31]:
f = open("blogs.txt", "r")
lines = f.readlines()

In [4]:
def get_root(s):
    s = s.strip()
    s = s.strip('/')
    return s

In [5]:
nodes = []
for l in lines:
    if l.startswith("\t") or l.startswith("["):
        continue
    n = get_root(l)
    if not n in nodes:
        nodes.append(n)
nodes_rev = dict(zip(nodes,range(0,len(nodes))))

In [93]:
# for (i,n1) in enumerate(nodes):
#     for n2 in nodes[i+1:-1]:
#         if lev(n1,n2) < 5:
#             print(n1, n2)

In [6]:
g = ig.Graph(directed=True)
g.add_vertices(len(nodes))
g.vs["label"] = nodes
current_node = 0
for l in lines:
    if l.startswith("["):
        continue
    if l.startswith("\t"):
        g.add_edges([(current_node,nodes_rev[get_root(l)])])
    else:
        current_node = nodes_rev[get_root(l)]
ig.summary(g)
g0 = copy.deepcopy(g)

IGRAPH D--- 2271 3146 -- 
+ attr: label (v)


In [34]:
len(g.vs)

2271

In [10]:
to_delete_ids = [v.index for v in g.vs if len(g.neighbors(v,mode="in"))==1 and len(g.neighbors(v,mode="out"))==0]
[print(g.vs[i]) for i in to_delete_ids]
g.delete_vertices(to_delete_ids)
ig.summary(g)

IGRAPH D--- 450 1325 -- 
+ attr: label (v)


In [288]:
part = leidenalg.find_partition(g, leidenalg.RBConfigurationVertexPartition, resolution_parameter = 1.0)
# part = leidenalg.find_partition(g, leidenalg.RBConfigurationVertexPartition, resolution_parameter = 1.0)
part_dic = dict()
for (pn,p) in enumerate(part):
    for i in p:
        part_dic[i] = pn
[print(len(p)) for p in part];
print(len(part))
# print(part_dic)

125
78
75
72
71
18
7
3
1
9


In [289]:
nodes_v = []
links_v = []
for (i,v) in enumerate(g.vs):
    nodes_v.append({
        "id": v["label"],
        "group": part_dic[i],
        "size": len(g.neighbors(v,mode="in"))
    })
    outn = g.neighbors(v,mode="out")
    for on in outn:
        links_v.append({
            "source": v["label"],
            "target": g.vs[on]["label"],
            "value": 1,
        })
d = {
    "nodes": nodes_v,
    "links": links_v
}
with open('t3.json', 'w') as outfile:
    json.dump(d, outfile, indent=2)

In [ ]:
# G = ig.Graph.Famous('Zachary')
# partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)
# ig.plot(part)

igraph.Vertex(<igraph.Graph object at 0x7ff4512794f0>, 0, {'name': 'a'})

In [36]:
g.vs[0]

igraph.Vertex(<igraph.Graph object at 0x7ff49e956e50>, 0, {'label': 'astralcodexten.substack.com'})

In [290]:
before_todo = True
nodes_done = [False for _ in nodes]
for l in lines:
    if before_todo:
        if l.startswith("[TODO]"):
            before_todo = False
            continue
        elif l.startswith("["):
            continue
        elif l.startswith("\t"):
            continue
        else:
            n = get_root(l)
            nodes_done[nodes_rev[n]] = True
            

In [252]:
ig.summary(g0)

IGRAPH D--- 2239 3081 -- 
+ attr: label (v)


In [264]:
N = [n for n,d in zip(nodes,nodes_done) if not d]
scores = [len(g0.neighbors(i,mode="in")) for (i,d) in zip(range(len(nodes)), nodes_done) if not d]

s_sort = np.flip(np.argsort(scores))

[print(scores[s],N[s]) for s in s_sort];
# [print(N[s]) for s in s_sort];
# [print(scores[s]) for s in s_sort];


5 nakedcapitalism.com
4 worthwhile.typepad.com
4 angrybearblog.com
4 professorbainbridge.com
4 braddelong.substack.com
4 uneasymoney.com
3 talkleft.com
3 rantburg.com
3 rightwingnews.com
3 rogerlsimon.com
3 offsettingbehaviour.blogspot.com
3 blogs.news.com.au/dailytelegraph/timblair
3 balkin.blogspot.com
3 lileks.com/bleats
3 freakonomics.com/blog
3 mises.org
3 christophe-barraud.com/en/blog
3 dailysignal.com
3 lawyersgunsmoneyblog.com
3 truthonthemarket.com
3 danieldrezner.com/blog
3 gnxp.com
3 economist.com/blogs/freeexchange
3 econospeak.blogspot.com
3 krugman.blogs.nytimes.com
3 mainlymacro.blogspot.com
3 newmonetarism.blogspot.com
3 ritholtz.com/blog
3 fxdiebold.blogspot.com
3 terrytao.wordpress.com
3 davegiles.blogspot.com
3 newsmax.com
3 io9.com
3 economicsone.com
3 stumblingandmumbling.typepad.com/stumbling_and_mumbling
3 streetwiseprofessor.com
3 math.columbia.edu/~woit/wordpress
3 fatasmihov.blogspot.com
3 taxprof.typepad.com/taxprof_blog
3 theothermccain.com
3 gatestoneinsti

In [28]:
f = open("blogs2.txt", "r")
lines = f.readlines()

nodes = []
for l in lines:
    n = get_root(l)
    if not n in nodes:
        nodes.append(n)
nodes_rev = dict(zip(nodes,range(0,len(nodes))))

comp_nodes = []
for l in lines:
    if l.startswith("\t"):
        continue
    n = get_root(l)
    if not n in comp_nodes:
        comp_nodes.append(n)

g = ig.Graph(directed=True)
g.add_vertices(len(nodes))
g.vs["label"] = nodes
current_node = 0
for l in lines:
    if l.startswith("\t"):
        g.add_edges([(current_node,nodes_rev[get_root(l)])])
    else:
        current_node = nodes_rev[get_root(l)]
ig.summary(g)
g0 = copy.deepcopy(g)

with open('blogs_done.txt', 'w') as f:
    for v in g0.vs:
        if v["label"] in comp_nodes:
            print("[" + v["label"] + "]", file=f)
            for no in g0.neighbors(v,mode="out"):
                print(g0.vs[no]["label"], file=f)

AN = []
SC = []
with open('blogs_todo.txt', 'w') as f:
    for v in g0.vs:
        if v["label"] not in comp_nodes:
            AN.append(v["label"])
            SC.append(len(g0.neighbors(v,mode="in")))
    s_sort = np.flip(np.argsort(SC))
    for s in s_sort:
        print("[" + AN[s] + "]", file=f)

IGRAPH D--- 2271 3146 -- 
+ attr: label (v)


In [29]:
SC[s_sort[0]]

4

In [20]:
comp_nodes

['astralcodexten.substack.com',
 'newscience.org',
 'applieddivinitystudies.com',
 'nintil.com',
 'westhunt.wordpress.com',
 'putanumonit.com',
 'rationallyspeakingpodcast.org',
 'vox.com/authors/kelsey-piper',
 'lesswrong.com',
 'marginalrevolution.com',
 'razib.substack.com',
 'scottaaronson.com/blog',
 'infoproc.blogspot.com',
 'zeynep.substack.com',
 'thezvi.wordpress.com',
 'elidourado.com/blog',
 'bayesianinvestor.com/blog',
 'scholars-stage.blogspot.com',
 'blogs.sciencemag.org/pipeline',
 'forum.effectivealtruism.org',
 'strangeloopcanon.com',
 'slimemoldtimemold.com',
 'freddiedeboer.substack.com',
 'beeminder.com',
 'cold-takes.com',
 'goodoptics.wordpress.com',
 'dynomight.net',
 'fantasticanachronism.com',
 'forourposterity.com',
 'arnoldkling.com/blog',
 'alltrades.substack.com',
 'bloomberg.com/view/contributors/AS6n2t3d_iA/tyler-cowen',
 'cafehayek.com',
 'mungowitzend.blogspot.com',
 'blog.supplysideliberal.com',
 'conversableeconomist.blogspot.com',
 'austrianeconomist